In [ ]:
%pylab notebook
from __future__ import print_function
from charistools.convertors import ET2Hypsometry
from charistools.hypsometry import Hypsometry
from charistools.modelEnv import ModelEnv
import datetime as dt
import matplotlib.pyplot as plt
import os

In [ ]:
configFile = '/Users/brodzik/ipython_notebooks/charis/et_modelEnv_config.ini'
myEnv = ModelEnv(tileConfigFile=configFile)
#basins = ['AM_Vakhsh_at_Komsomolabad',
#          'GA_Karnali_at_Benighat',
#          'GA_Langtang_at_Kyanjin',
#          'GA_Narayani_at_Devghat',
#          'GA_SaptaKosi_at_Chatara',
#          'IN_Hunza_at_DainyorBridge']
ids = ['AM', 'BR', 'IN_v01', 'GA_v01', 'SY_v01']


In [ ]:
fullbasins = ["%s_fullbasinmasks" % id for id in ids]
snowybasins = ["%s_snow_fullbasinmasks" % id for id in ids]

In [ ]:
basins = snowybasins + fullbasins
basins[7:]


In [ ]:
for basin in basins[7:]:
    outfile = myEnv.hypsometry_filename(
        type='et_by_elevation',
        drainageID=basin,
        et_source='mod16')
    by_elev = ET2Hypsometry(drainageID=basin,
                            modelEnv=myEnv,
                            start_yyyymm=200101,
                            stop_yyyymm=201412,
                            decimal_places=9,
                            outfile=outfile,
                            verbose=True)

In [ ]:
basin='SY_Naryn_at_NarynTown'
outfile = myEnv.hypsometry_filename(
        type='et_by_elevation',
        drainageID=basin,
        et_source='mod16')
#by_elev = ET2Hypsometry(drainageID=basin,
#                        modelEnv=myEnv,
#                        start_yyyymm=200101,
#                        stop_yyyymm=201412,
#                        decimal_places=9,
#                        outfile=outfile,
#                        verbose=True)
outfile

In [ ]:
by_elev = Hypsometry(outfile)

In [ ]:
by_elev.data

In [ ]:
# Collapse the ET by elevation into just a time series by date:
monthlyET_file = "/Users/brodzik/projects/CHARIS/evapotranspiration/mod16/%s.mod16.ET.csv" % basin
monthlyET_file

In [ ]:
by_elev.data_by_doy()

In [ ]:

comments=["Time Series created : " + str(dt.datetime.now()),
          "MOD16 ET for drainageID : %s" % basin,
          "Units : km^3",
          "Hypsometry used to derive this time series: %s" % os.path.basename(outfile),
          "COLUMNS: Year Month Day DOY ET"]
f = open(monthlyET_file, "w")
for line in comments:
    f.write("# %s\n" % line)
for key,value in by_elev.data_by_doy().iteritems():
    print("%4d %2d %2d %3d %.9f" % (key.year, key.month, key.day, key.dayofyear, value),
          file=f)
f.close()

In [ ]:
fig, ax = plt.subplots(1)
by_elev.data_by_doy().plot(label="from ET_by_elevation")
ax.legend()
ax.set_title("%s ET" % basin)

In [ ]:
fig, ax = plt.subplots(1)
by_elev.imshow(ax=ax, title=basin + ' ET', cmap='Greens_r', 
               xlabel='Date', dateFormat='%b', ylabel='Elevation ($m$)',
               vmax=0.049)

In [ ]:
# Read the annual file that Karl produced before
from charistools.timeSeries import TimeSeries
origfile = '/Users/brodzik/projects/CHARIS/evapotranspiration/mod16/IN_Hunza_at_DainyorBridge.mod16.ET.csv'
origts = TimeSeries(origfile)

In [ ]:
fig, ax = plt.subplots(1)
origts.data['et'].plot(label="Karl's ET")
by_elev.data_by_doy().plot(label="from ET_by_elevation")
ax.legend()
ax.set_title("%s ET" % basin)
plt.savefig("/Users/brodzik/projects/CHARIS/evapotranspiration/mod16/Hunza_ET_orig_vs_by_elev.png")

In [ ]:
from charistools.timeSeries import TimeSeries
et = TimeSeries(monthlyET_file)
print(et.comments)
print(et.data)